<a href="https://colab.research.google.com/github/theCodeNilesh/operating_vehicles_fogg/blob/main/Operating_vehicles_fogg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ultralytics/yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def record_video(filename='video.mp4'):
    js = Javascript("""
    async function recordVideo() {
        const options = { mimeType: "video/webm; codecs=vp9" };
        const div = document.createElement('div');
        const capture = document.createElement('button');
        const stopCapture = document.createElement("button");
        capture.textContent = "Start Recording";
        capture.style.background = "green";
        capture.style.color = "white";
        stopCapture.textContent = "Stop Recording";
        stopCapture.style.background = "red";
        stopCapture.style.color = "white";
        div.appendChild(capture);
        const video = document.createElement('video');
        const recordingVid = document.createElement("video");
        video.style.display = 'block';
        const stream = await navigator.mediaDevices.getUserMedia({ video: true });
        let recorder = new MediaRecorder(stream, options);
        document.body.appendChild(div);
        div.appendChild(video);
        video.srcObject = stream;
        await video.play();
        google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
        await new Promise((resolve) => { capture.onclick = resolve; });
        recorder.start();
        capture.replaceWith(stopCapture);
        await new Promise((resolve) => stopCapture.onclick = resolve);
        recorder.stop();
        let recData = await new Promise((resolve) => recorder.ondataavailable = resolve);
        let arrBuff = await recData.data.arrayBuffer();
        stream.getVideoTracks()[0].stop();
        div.remove();
        let binaryString = "";
        let bytes = new Uint8Array(arrBuff);
        bytes.forEach((byte) => { binaryString += String.fromCharCode(byte); })
        return btoa(binaryString);
    }
    """)

    try:
        display(js)
        data = eval_js('recordVideo({})')
        binary = b64decode(data)
        with open(filename, "wb") as video_file:
            video_file.write(binary)
        print(f"Finished recording video. Saved binary under filename in current working directory: {filename}")
    except Exception as err:
        print(str(err))
        return filename

# Run the function, get the video path as saved in your notebook, and play it back here.
video_path = record_video()

<IPython.core.display.Javascript object>

Finished recording video. Saved binary under filename in current working directory: video.mp4


In [ ]:
%cd yolov5

/content/yolov5


In [ ]:
!pip install -r requirements.txt

In [ ]:
import cv2
import numpy as np

In [ ]:
def DarkChannel(im, sz):
    b, g, r = cv2.split(im)
    dc = cv2.min(cv2.min(r, g), b)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (sz, sz))
    dark = cv2.erode(dc, kernel)
    return dark

In [ ]:
def AtmLight(img, dark):
    A = np.percentile(img, 99.9, axis=(0, 1))  # Adjust percentile
    return A

In [ ]:
def TransmissionEstimate(im, A, sz):
    omega = 0.92  # Adjust weight
    im3 = np.empty(im.shape, im.dtype)

    for ind in range(0, 3):
        A_channel = A[ind]
        im3[:, :, ind] = im[:, :, ind] / A_channel

    transmission = 1 - omega * DarkChannel(im3, sz)
    return transmission

In [ ]:
def Guidedfilter(I, p, r, eps):
    mean_I = cv2.boxFilter(I, cv2.CV_64F, (r, r))
    mean_p = cv2.boxFilter(p, cv2.CV_64F, (r, r))
    mean_Ip = cv2.boxFilter(I * p, cv2.CV_64F, (r, r))
    cov_Ip = mean_Ip - mean_I * mean_p

    mean_II = cv2.boxFilter(I * I, cv2.CV_64F, (r, r))
    var_I = mean_II - mean_I * mean_I

    a = cov_Ip / (var_I + eps)
    b = mean_p - a * mean_I

    mean_a = cv2.boxFilter(a, cv2.CV_64F, (r, r))
    mean_b = cv2.boxFilter(b, cv2.CV_64F, (r, r))

    q = mean_a * I + mean_b
    return q

In [ ]:
def TransmissionRefine(im, et):
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    r = 50  # radius of filter
    eps = 0.0001  # regularization parameter
    t = Guidedfilter(gray, et, r, eps)
    return t

In [ ]:
def Recover(im, t, A, tx=0.1):
    res = np.empty(im.shape, im.dtype)
    t = cv2.max(t, tx)

    for ind in range(0, 3):
        res[:, :, ind] = (im[:, :, ind] - A[ind]) / t + A[ind]

    return res.clip(0, 1)  #

In [ ]:
def enhance_quality(frame):
    # Gamma correction
    gamma = 1.05
    enhanced_frame = np.clip(((frame * 255 / 255) ** gamma) * 255, 0, 255).astype(np.uint8)

    return enhanced_frame

In [ ]:
video_path = "/content/video.mp4"

output_video_path = "/content/Output_sample.mp4"

In [ ]:
cap = cv2.VideoCapture(video_path)

In [ ]:
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

In [ ]:
# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

In [ ]:
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    I = frame.astype('float64') / 255

    dark = DarkChannel(I, 20)  # Adjusted window size
    A = AtmLight(I, dark)
    te = TransmissionEstimate(I, A, 20)  # Adjusted window size
    t = TransmissionRefine(frame, te)
    J = Recover(I, t, A, 0.05)  # Adjusted tx parameter for more haze removal

    # Enhance quality and clarity
    J = enhance_quality(J)

    # Write the processed frame to the output video
    out.write(J)


In [ ]:
cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
!python detect.py --source /content/Output_sample.mp4

detect: weights=yolov5s.pt, source=/content/Output_sample.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-304-g22361691 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

100% 14.1M/14.1M [00:00<00:00, 203MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded
video 1/1 (1/132) /content/Output_sample.mp4: 480x640 1 person, 52.0ms
video 1/1 (2/132) /content/Output_sample.mp4: 480x640 1 person, 9.5ms
video 1/1 (3/132) /content/Output_sample.mp4: 480x640 1 person, 9.4ms
video 1/1 (4/132) /content/Output_sample.mp4: 480x

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
